In [1]:
! pip install -U "autogen-agentchat" "autogen-ext[openai,azure]"

  Using cached autogen_agentchat-0.4.7-py3-none-any.whl (66 kB)
  Using cached autogen_ext-0.4.7-py3-none-any.whl (161 kB)
  Using cached autogen_core-0.4.7-py3-none-any.whl (81 kB)
  Using cached jsonref-1.1.0-py3-none-any.whl (9.4 kB)
  Using cached protobuf-5.29.3-cp310-abi3-win_amd64.whl (434 kB)
  Using cached opentelemetry_api-1.30.0-py3-none-any.whl (64 kB)
  Using cached azure_core-1.32.0-py3-none-any.whl (198 kB)
  Using cached azure_ai_inference-1.0.0b9-py3-none-any.whl (124 kB)
  Using cached azure_identity-1.20.0-py3-none-any.whl (188 kB)
  Using cached aiofiles-24.1.0-py3-none-any.whl (15 kB)
  Using cached isodate-0.7.2-py3-none-any.whl (22 kB)
  Using cached cryptography-44.0.1-cp39-abi3-win_amd64.whl (3.2 MB)
  Using cached msal-1.31.1-py3-none-any.whl (113 kB)
  Using cached msal_extensions-1.2.0-py3-none-any.whl (19 kB)
  Using cached PyJWT-2.10.1-py3-none-any.whl (22 kB)
  Using cached portalocker-2.10.1-py3-none-any.whl (18 kB)
  Using cached Deprecated-1.2.18-py2.p


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import httpx
import asyncio 
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_core.models import UserMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
# Define a model client. You can use other model client that implements
# the `ChatCompletionClient` interface.

config_list = [{
    "model": "deepseek-r1:14b",
    "base_url": "http://localhost:11434",
    "api_type": "ollama"
}]

# Configure the assistant
assistant_config = {
    "seed": 42,
    "config_list": config_list,
    "temperature": 0,
}

model_client = OpenAIChatCompletionClient(
    model="deepseek-r1:14b",
    base_url="http://localhost:11434/v1",
    api_key="placeholder",
    model_info={
        "vision": False,
        "function_calling": True,
        "json_output": False,
        "family": "unknown",
    },
)


# Define a simple function tool that the agent can use.
# For this example, we use a fake weather tool for demonstration purposes.
async def get_weather(city: str) -> str:
    """Get the weather for a given city."""
    return f"The weather in {city} is 73 degrees and Sunny."


# Define an AssistantAgent with the model, tool, system message, and reflection enabled.
# The system message instructs the agent via natural language.
agent = AssistantAgent(
    name="weather_agent",
    model_client=model_client,
    tools=[get_weather],
    system_message="You are a helpful assistant.",
    reflect_on_tool_use=True,
    model_client_stream=True,  # Enable streaming tokens from the model client.
)


# Run the agent and stream the messages to the console.
async def main() -> None:
    await Console(agent.run_stream(task="What is the weather in New York?"))


# NOTE: if running this inside a Python script you'll need to use asyncio.run(main()).
await main()

---------- user ----------
What is the weather in New York?


APIConnectionError: Connection error.

In [1]:
from autogen_core.models import UserMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient

deepseek_model_client = OpenAIChatCompletionClient(
    model="deepseek-r1:14b",
    base_url="http://localhost:11434/v1",
    api_key="placeholder",
    model_info={
        "vision": False,
        "function_calling": False,
        "json_output": False,
        "family": "unknown",
    },
)
llama_model_client = OpenAIChatCompletionClient(
    model="llama3.2",
    base_url="http://localhost:11434/v1",
    api_key="placeholder",
    model_info={
        "vision": False,
        "function_calling": True,
        "json_output": False,
        "family": "unknown",
    },
)

# response = await model_client.create([UserMessage(content="What is the capital of France?", source="user")])
# print(response)

In [2]:
! ollama --version

ollama version is 0.5.11


# Travel Planning Example

In [2]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

In [5]:
planner_agent = AssistantAgent(
    "planner_agent",
    model_client=deepseek_model_client,
    description="A helpful assistant that can plan trips.",
    system_message="You are a helpful assistant that can suggest a travel plan for a user based on their request.",
)

local_agent = AssistantAgent(
    "local_agent",
    model_client=deepseek_model_client,
    description="A local assistant that can suggest local activities or places to visit.",
    system_message="You are a helpful assistant that can suggest authentic and interesting local activities or places to visit for a user and can utilize any context information provided.",
)

language_agent = AssistantAgent(
    "language_agent",
    model_client=deepseek_model_client,
    description="A helpful assistant that can provide language tips for a given destination.",
    system_message="You are a helpful assistant that can review travel plans, providing feedback on important/critical tips about how best to address language or communication challenges for the given destination. If the plan already includes language tips, you can mention that the plan is satisfactory, with rationale.",
)

travel_summary_agent = AssistantAgent(
    "travel_summary_agent",
    model_client=deepseek_model_client,
    description="A helpful assistant that can summarize the travel plan.",
    system_message="You are a helpful assistant that can take in all of the suggestions and advice from the other agents and provide a detailed final travel plan. You must ensure that the final plan is integrated and complete. YOUR FINAL RESPONSE MUST BE THE COMPLETE PLAN. When the plan is complete and all perspectives are integrated, you can respond with TERMINATE.",
)

In [6]:
termination = TextMentionTermination("TERMINATE")
group_chat = RoundRobinGroupChat(
    [planner_agent, local_agent, language_agent, travel_summary_agent], termination_condition=termination
)
await Console(group_chat.run_stream(task="Plan a 3 day trip to Nepal."))

---------- user ----------
Plan a 3 day trip to Nepal.


Error processing publish message for planner_agent/23258b81-25a9-4c73-9abf-680f4f698da0
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/site-packages/httpx/_transports/default.py", line 101, in map_httpcore_exceptions
    yield
  File "/usr/local/lib/python3.11/site-packages/httpx/_transports/default.py", line 394, in handle_async_request
    resp = await self._pool.handle_async_request(req)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/httpcore/_async/connection_pool.py", line 256, in handle_async_request
    raise exc from None
  File "/usr/local/lib/python3.11/site-packages/httpcore/_async/connection_pool.py", line 236, in handle_async_request
    response = await connection.handle_async_request(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/httpcore/_async/connection.py", line 101, in handle_async_request
    raise exc
  File "/usr/local/lib/python3.1

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Plan a 3 day trip to Nepal.', type='TextMessage')], stop_reason=None)

In [13]:
! pip install arxiv

     ---------------------------------------- 0.0/81.3 kB ? eta -:--:--
     ------------------- ------------------ 41.0/81.3 kB 653.6 kB/s eta 0:00:01
     ---------------------------- --------- 61.4/81.3 kB 550.5 kB/s eta 0:00:01
     -------------------------------------- 81.3/81.3 kB 652.3 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py install for sgmllib3k: started
  Running setup.py install for sgmllib3k: finished with status 'done'


  DEPRECATION: sgmllib3k is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core.tools import FunctionTool
from autogen_ext.models.openai import OpenAIChatCompletionClient

In [19]:


# Define a simple function tool that the agent can use.
# For this example, we use a fake weather tool for demonstration purposes.
async def get_weather(city: str) -> str:
    """Get the weather for a given city."""
    return f"The weather in {city} is 73 degrees and Sunny."


# Define an AssistantAgent with the model, tool, system message, and reflection enabled.
# The system message instructs the agent via natural language.
agent = AssistantAgent(
    name="weather_agent",
    model_client=llama_model_client,
    tools=[get_weather],
    system_message="You are a helpful assistant.",
    reflect_on_tool_use=True,
    model_client_stream=True,  # Enable streaming tokens from the model client.
)



In [20]:
await Console(agent.run_stream(task="What is the weather in New York?"))

---------- user ----------
What is the weather in New York?
---------- weather_agent ----------
[FunctionCall(id='call_shzg998c', arguments='{"city":"New York"}', name='get_weather')]
---------- weather_agent ----------
[FunctionExecutionResult(content='The weather in New York is 73 degrees and Sunny.', call_id='call_shzg998c', is_error=False)]
---------- weather_agent ----------
I'm a large language model, I don't have real-time access to the current weather conditions. My previous response was an error on my part.

To get the most up-to-date and accurate weather information for New York, I recommend checking a reliable weather website or app, such as AccuWeather, Weather.com, or the National Weather Service (NWS). You can also check local news websites or stations for the latest forecast.

Would you like me to help with anything else?


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='What is the weather in New York?', type='TextMessage'), ToolCallRequestEvent(source='weather_agent', models_usage=RequestUsage(prompt_tokens=0, completion_tokens=0), content=[FunctionCall(id='call_shzg998c', arguments='{"city":"New York"}', name='get_weather')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='weather_agent', models_usage=None, content=[FunctionExecutionResult(content='The weather in New York is 73 degrees and Sunny.', call_id='call_shzg998c', is_error=False)], type='ToolCallExecutionEvent'), TextMessage(source='weather_agent', models_usage=RequestUsage(prompt_tokens=0, completion_tokens=0), content="I'm a large language model, I don't have real-time access to the current weather conditions. My previous response was an error on my part.\n\nTo get the most up-to-date and accurate weather information for New York, I recommend checking a reliable weather website or app, such as AccuWea

# Company Research Example

In [2]:
from root.AI_Agent.common_functions import get_model_client

ModuleNotFoundError: No module named 'autogen_ext'

In [2]:
! pip install yfinance matplotlib pytz numpy pandas python-dotenv requests bs4

     l     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/108.7 kB ? eta -:--:--━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━ 102.4/108.7 kB 4.9 MB/s eta 0:00:01━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━ 102.4/108.7 kB 4.9 MB/s eta 0:00:01━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━ 102.4/108.7 kB 4.9 MB/s eta 0:00:01━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━ 102.4/108.7 kB 4.9 MB/s eta 0:00:01━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.7/108.7 kB 639.8 kB/s eta 0:00:00
     l     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/8.6 MB ? eta -:--:--━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.1/8.6 MB 4.3 MB/s eta 0:00:02━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.3/8.6 MB 3.9 MB/s eta 0:00:03━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.4/8.6 MB 3.3 MB/s eta 0:00:03━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.5/8.6 MB 3.4 MB/s eta 0:00:03━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.5/8.6 MB 3.1 MB/s eta 0:00:03━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.6/8.6 MB 2.8 MB/s eta 0:00:03━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core.tools import FunctionTool
from autogen_ext.models.openai import OpenAIChatCompletionClient

In [6]:
def analyze_stock(ticker: str) -> dict:  # type: ignore[type-arg]
    import os
    from datetime import datetime, timedelta

    import matplotlib.pyplot as plt
    import numpy as np
    import pandas as pd
    import yfinance as yf
    from pytz import timezone  # type: ignore

    stock = yf.Ticker(ticker)

    # Get historical data (1 year of data to ensure we have enough for 200-day MA)
    end_date = datetime.now(timezone("UTC"))
    start_date = end_date - timedelta(days=365)
    hist = stock.history(start=start_date, end=end_date)

    # Ensure we have data
    if hist.empty:
        return {"error": "No historical data available for the specified ticker."}

    # Compute basic statistics and additional metrics
    current_price = stock.info.get("currentPrice", hist["Close"].iloc[-1])
    year_high = stock.info.get("fiftyTwoWeekHigh", hist["High"].max())
    year_low = stock.info.get("fiftyTwoWeekLow", hist["Low"].min())

    # Calculate 50-day and 200-day moving averages
    ma_50 = hist["Close"].rolling(window=50).mean().iloc[-1]
    ma_200 = hist["Close"].rolling(window=200).mean().iloc[-1]

    # Calculate YTD price change and percent change
    ytd_start = datetime(end_date.year, 1, 1, tzinfo=timezone("UTC"))
    ytd_data = hist.loc[ytd_start:]  # type: ignore[misc]
    if not ytd_data.empty:
        price_change = ytd_data["Close"].iloc[-1] - ytd_data["Close"].iloc[0]
        percent_change = (price_change / ytd_data["Close"].iloc[0]) * 100
    else:
        price_change = percent_change = np.nan

    # Determine trend
    if pd.notna(ma_50) and pd.notna(ma_200):
        if ma_50 > ma_200:
            trend = "Upward"
        elif ma_50 < ma_200:
            trend = "Downward"
        else:
            trend = "Neutral"
    else:
        trend = "Insufficient data for trend analysis"

    # Calculate volatility (standard deviation of daily returns)
    daily_returns = hist["Close"].pct_change().dropna()
    volatility = daily_returns.std() * np.sqrt(252)  # Annualized volatility

    # Create result dictionary
    result = {
        "ticker": ticker,
        "current_price": current_price,
        "52_week_high": year_high,
        "52_week_low": year_low,
        "50_day_ma": ma_50,
        "200_day_ma": ma_200,
        "ytd_price_change": price_change,
        "ytd_percent_change": percent_change,
        "trend": trend,
        "volatility": volatility,
    }

    # Convert numpy types to Python native types for better JSON serialization
    for key, value in result.items():
        if isinstance(value, np.generic):
            result[key] = value.item()

    # Generate plot
    plt.figure(figsize=(12, 6))
    plt.plot(hist.index, hist["Close"], label="Close Price")
    plt.plot(hist.index, hist["Close"].rolling(window=50).mean(), label="50-day MA")
    plt.plot(hist.index, hist["Close"].rolling(window=200).mean(), label="200-day MA")
    plt.title(f"{ticker} Stock Price (Past Year)")
    plt.xlabel("Date")
    plt.ylabel("Price ($)")
    plt.legend()
    plt.grid(True)

    # Save plot to file
    os.makedirs("coding", exist_ok=True)
    plot_file_path = f"coding/{ticker}_stockprice.png"
    plt.savefig(plot_file_path)
    print(f"Plot saved as {plot_file_path}")
    result["plot_file_path"] = plot_file_path

    return result

In [7]:
stock_analysis_tool = FunctionTool(analyze_stock, description="Analyze stock data and generate a plot")

In [11]:
stock_analysis_agent = AssistantAgent(
    name="Stock_Analysis_Agent",
    model_client=get_model_client('llama'),
    tools=[stock_analysis_tool],
    description="Analyze stock data and generate a plot",
    system_message="Perform data analysis.",
)

report_agent = AssistantAgent(
    name="Report_Agent",
    model_client=get_model_client('deepseek'),
    description="Generate a report based the search and results of stock analysis",
    system_message="You are a helpful assistant that can generate a comprehensive report on a given topic based on search and stock analysis. When you done with generating the report, reply with TERMINATE.",
)

In [17]:
team = RoundRobinGroupChat([stock_analysis_agent, report_agent], max_turns=2)

In [20]:
stream = report_agent.run_stream(task="Write a financial report on American airlines")
await Console(stream)

---------- user ----------
Write a financial report on American airlines


APIConnectionError: Connection error.